importing libraries and Loading Data

In [1]:

import nltk
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle
import datetime
nltk.download("punkt")
nltk.download("wordnet")

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!
Instructions for updating:
Colocations handled automatically by placer.


[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [ ]:
#loading data
with open(r"C:\Users\ayush\project_2year\chat_bot_3\intents.json") as file:
    data = json.load(file)

In [ ]:
seat_count = 50

In [ ]:
#intializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
#looping through our data
for intent in data['intents']:
    for pattern in intent['patterns']:
        pattern = pattern.lower()
        #creating a list of words
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
#stemming
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [ ]:
#vectorization
training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)

In [ ]:
#converting training data into numpy arraays
training = np.array(training)
output = np.array(output)

In [ ]:
#saving data to disk
with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [ ]:
#bulding a model
tf.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

In [ ]:
#preprocessing input

def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(w) for w in tokens]
    return tokens

In [ ]:
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for x, word in enumerate(vocab):
            if word == w:
                bow[x] = 1
    return np.array(bow)

In [ ]:
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[index, res] for index, res in enumerate(result) if res > thresh]
    y_pred.sort(key = lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

In [ ]:
def get_response(intents_list, intents_json):
    global seat_count 
    if len(intents_list) == 0:
        result = "Sorry! I don't understood."
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]

        # book a table 
        if (tag =="book_table"):
            t = input("at which time(hh:mm PM):-")
            tt = t.split(':')[0]
            if(tt>=4 and tt<12):
                seat_count = seat_count - 1
                result = "Your table has been booked successfully. time is " + t + ". Remaining tables: " + str(seat_count)

            else:
                result = "please check timing of resturant. Thanks!!"


        elif tag == "available_tables":
            result = "There are " + str(seat_count) + " tables available at the moment."

        # menu    
        elif tag == "menu":
             day = datetime.datetime.now()
             day = day.strftime("%A")

             if day == "Monday":
                    result = "Chef recommends: Steamed Tofu with Schezwan Peppercorn, Eggplant with Hot Garlic Sauce, Chicken & Chives, Schezwan Style, Diced Chicken with Dry Red Chilli, Schezwan Pepper"      
             elif day == "Tuesday":
                result = "Chef recommends: Asparagus Fresh Shitake & King Oyster Mushroom, Stir Fried Chilli Lotus Stem, Crispy Fried Chicken with Dry Red Pepper, Osmanthus Honey, Hunan Style Chicken"
             elif day == "Wednesday":
                result = "Chef recommends: Baby Pokchoi Fresh Shitake Shimeji Straw & Button Mushroom, Mock Meat in Hot Sweet Bean Sauce, Diced Chicken with Bell Peppers & Onions in Hot Garlic Sauce, Chicken in Chilli Black Bean & Soy Sauce"
             elif day == "Thursday":
                result = "Chef recommends: Eggplant & Tofu with Chilli Oyster Sauce, Corn, Asparagus Shitake & Snow Peas in Hot Bean Sauce, Diced Chicken Plum Honey Chilli Sauce, Clay Pot Chicken with Dried Bean Curd Sheet"
             elif day == "Friday":
                result = "Chef recommends: Kailan in Ginger Wine Sauce, Tofu with Fresh Shitake & Shimeji, Supreme Soy Sauce, Diced Chicken in Black Pepper Sauce, Sliced Chicken in Spicy Mala Sauce"
             elif day == "Saturday":
                result = "Chef recommends: Kung Pao Potato, Okra in Hot Bean Sauce, Chicken in Chilli Black Bean & Soy Sauce, Hunan Style Chicken"
             elif day == "Sunday":
                 result = "Chef recommends: Stir Fried Bean Sprouts & Tofu with Chives, Vegetable Thou Sou, Diced Chicken Plum Honey Chilli Sauce, Diced Chicken in Black Pepper Sauce"  

        # offers
        elif tag == "offers":
             day = datetime.datetime.now()
             day = day.strftime("%A")

             if day == "Monday": 
                    result = "Today Offer: 10% off above ₹200 order"      
             elif day == "Tuesday":
                result = "Chef recommends: Asparagus Fresh Shitake & King Oyster Mushroom, Stir Fried Chilli Lotus Stem, Crispy Fried Chicken with Dry Red Pepper, Osmanthus Honey, Hunan Style Chicken"
             elif day == "Wednesday":
                result = "Chef recommends: Baby Pokchoi Fresh Shitake Shimeji Straw & Button Mushroom, Mock Meat in Hot Sweet Bean Sauce, Diced Chicken with Bell Peppers & Onions in Hot Garlic Sauce, Chicken in Chilli Black Bean & Soy Sauce"
             elif day == "Thursday":
                result = "Chef recommends: Eggplant & Tofu with Chilli Oyster Sauce, Corn, Asparagus Shitake & Snow Peas in Hot Bean Sauce, Diced Chicken Plum Honey Chilli Sauce, Clay Pot Chicken with Dried Bean Curd Sheet"
             elif day == "Friday":
                result = "Chef recommends: Kailan in Ginger Wine Sauce, Tofu with Fresh Shitake & Shimeji, Supreme Soy Sauce, Diced Chicken in Black Pepper Sauce, Sliced Chicken in Spicy Mala Sauce"
             elif day == "Saturday":
                result = "Chef recommends: Kung Pao Potato, Okra in Hot Bean Sauce, Chicken in Chilli Black Bean & Soy Sauce, Hunan Style Chicken"
             elif day == "Sunday":
                 result = "deal of the day : 20% off above ₹500 order"            
        

        else:
            for i in list_of_intents:
                if i["tag"] == tag:
                    result = random.choice(i["responses"])
                    break

    return str(result)

In [ ]:
print("PRESS 0 if you don;t want to chat with me.")

while True:
    message = input("")
    if message == "0":
        break
    intents = pred_class(message, words, labels)
    result = get_response(intents, data)
    print(result)